---
title:  ID identification
subtitle: limited feature extraction / anomaly detection
---

There are couple of ways to identify the ID.

-   Variance method [@liuMagneticDiscontinuitiesSolar2022] : Large variance in the magnetic field compared with neighboring intervals (see [notebook](./detection/01_variance.ipynb))

-   Partial variance increment (PVI) method :
    
    -   @vaskoKineticscaleCurrentSheets2022

-   B-criterion [@burlagaTangentialDiscontinuitiesSolar1969] : a directional change of the magnetic ﬁeld larger than 30° during 60 s

-   TS-criterion [@tsurutaniInterplanetaryDiscontinuitiesTemporal1979] : $|ΔB|/|B| \geq 0.5$ within 3 minutes



Traditional methods (B-criterion and TS-criterion) rely on magnetic ﬁeld variations with a certain time lag. B-criterion has, as its main condition. In their methods, the IDs below the thresholds are artiﬁcially abandoned. Therefore, identiﬁcation criteria may affect the statistical results, and there is likely to be a discrepancy between the ﬁndings via B-criterion and TS- criterion.

In [ ]:
#| default_exp core/detection

In [ ]:
#| export
from datetime import timedelta
import polars as pl

from typing import overload

In [ ]:
#| export
# some helper functions
@overload
def pl_format_time(df: pl.DataFrame, tau: timedelta) -> pl.DataFrame: pass

@overload
def pl_format_time(df: pl.LazyFrame, tau: timedelta) -> pl.LazyFrame: pass

def pl_format_time(df: pl.LazyFrame, tau: timedelta):
    return df.with_columns(
        tstart=pl.col("time"),
        tstop=(pl.col("time") + tau),
        time=(pl.col("time") + tau / 2),
    )

## Pipelines

In [ ]:
# | export
from discontinuitypy.detection.variance import compute_indices, filter_indices

def detect_events(
    data: pl.LazyFrame,
    tau: timedelta,
    ts: timedelta,
    bcols,
    sparse_num=None,
    method="liu",
    **kwargs,
):
    indices = compute_indices(data, tau, bcols)
    if sparse_num is None:
        sparse_num = tau / ts // 3

    events = indices.pipe(filter_indices, sparse_num=sparse_num).pipe(
        pl_format_time, tau
    ).collect()

    return events